In [110]:
import gzip
import csv
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)

In [ ]:
df = pd.read_csv('raw_data/TenantInfo-and-usage_shuffled_inf.csv'#, nrows=1000
                )


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df_v =pd.read_csv('raw_data/verbatim.csv', sep='""', delimiter='\t', quotechar='"',
                  encoding='utf-16', error_bad_lines=False, skiprows=1, names=['TenantId','Comment', 'Theme'])

df_v.dropna(inplace=True)
df_v = df_v.loc[df_v['Theme'] !='null-0""']

In [ ]:
df_v.info()

In [ ]:
s = 'Commerce - Manage Subscriptions and Licenses (Add, Modify and Cancel)-0.172084481""'

In [ ]:
s_l = s.split('-')
print(s_l)
s_l[2]

In [ ]:
df_v_fake = df_v.copy()
df_v_fake.loc[df_v_fake['TenantId'] == '27299C40-0A78-4EAC-B101-DAD988D57031']
# df_v_fake['workload'] = ''
# df_v_fake['problem'] = ''
# df_v_fake['score'] = ''
for i, row in df_v_fake.iterrows():
    id_str = row.TenantId
    c_str = row.Comment
    t_str = row.Theme
    t = t_str[:-2]#.split('-')
    row.TenantId = id_str[1:-3]
    row.Comment = c_str[:-2]
    row.Theme = t
#     if len(t_list) == 3:
#         row.workload = t_list[0][:-1]
#         row.problem = t_list[1][1:]
#         row.score = t_list[2][:-2]
#     elif len(t_list) == 2:
#         row.workload = 'missing wl'
#         row.problem = t_list[0]
#         row.score = t_list[1][:-2]
#     else:
#         print(i)

In [ ]:
df_v_fake.head()

In [ ]:
tenid = list(df['TenantId'])
df_v_filter = df_v_fake.loc[df_v_fake['TenantId'].isin(tenid) & (df_v_fake['Comment'] != '') &
                            ~df_v_fake.Comment.str.contains(";&#")]

In [ ]:
df_v_filter.shape

In [75]:
len(df_v_filter['TenantId'].unique())

161725

In [76]:
df_v_sorted = df_v_filter.sort_values('TenantId')
df_v_sorted = df_v_sorted.reset_index()
df_v_sorted = df_v_sorted.drop('index', axis=1)

In [77]:
df_v_sorted.head()

,TenantId,Comment,Theme
0,00019767-249C-4208-9E68-4CFBBF916A71,great service,"Admin - Manage my users, groups and resources"
1,0001FB27-F238-4468-A6B4-E259F0A3C91A,He was great thank you,Skype - Setup and use Skype services
2,0001FCBE-E449-467F-B72F-9C9D2BFB924A,Maygan was very patient,Admin - Setup domain and DNS settings for Office 365
3,00029D24-E453-4E43-9C67-DAE820456AFF,Was a pleasure to work with and knowledgeable,"IP - Transport, Org Compliance and Org Security"
4,00029D24-E453-4E43-9C67-DAE820456AFF,Was a pleasure to work with and was knowledgeable,"IP - Transport, Org Compliance and Org Security"


In [78]:
df_v_try = df_v_sorted.copy()

df_v_try_1 = df_v_try.groupby('TenantId')['Comment'].apply(' '.join).reset_index()
df_v_try_2 = df_v_try.groupby('TenantId')['Theme'].apply(' '.join).reset_index()

df_v_com = df_v_try_1.merge(df_v_try_2, on='TenantId')

In [79]:
df_v_com.shape

(161725, 3)

In [80]:
df_v_com.loc[df_v_com['TenantId'] == '27299C40-0A78-4EAC-B101-DAD988D57031']

,TenantId,Comment,Theme


In [81]:
df_v_filter.loc[df_v_filter['Comment'] == '']

,TenantId,Comment,Theme


In [82]:
df_v_com.iloc[29700, 1]

'My experience was great, very helpfull employee and was patient and knew what he was doing, true professional!'

In [83]:
df_v_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161725 entries, 0 to 161724
Data columns (total 3 columns):
TenantId    161725 non-null object
Comment     161725 non-null object
Theme       161725 non-null object
dtypes: object(3)
memory usage: 9.9+ MB


In [84]:
df_v_com.head()

,TenantId,Comment,Theme
0,00019767-249C-4208-9E68-4CFBBF916A71,great service,"Admin - Manage my users, groups and resources"
1,0001FB27-F238-4468-A6B4-E259F0A3C91A,He was great thank you,Skype - Setup and use Skype services
2,0001FCBE-E449-467F-B72F-9C9D2BFB924A,Maygan was very patient,Admin - Setup domain and DNS settings for Office 365
3,00029D24-E453-4E43-9C67-DAE820456AFF,Was a pleasure to work with and knowledgeable Was a pleasure to work with and was knowledgeable,"IP - Transport, Org Compliance and Org Security IP - Transport, Org Compliance and Org Security"
4,0002D497-6351-416B-A000-80F7DAB22567,It was good to understand about CNAME,Keep mailboxes free of spam and viruses


## Join TenantInfo-and-usage_shuffled_inf.csv file and verbatim.csv file.

In [85]:
df.shape

(1865326, 228)

In [86]:
df_all  = df_v_com.merge(df, on='TenantId')

In [87]:
df_all.shape

(161725, 230)

In [88]:
df_all = df_all.sample(frac=1.0)

In [89]:
df_all[['Comment', 'Theme']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161725 entries, 67729 to 3968
Data columns (total 2 columns):
Comment    161725 non-null object
Theme      161725 non-null object
dtypes: object(2)
memory usage: 3.7+ MB


In [90]:
df_all.to_csv('raw_data/TenantInfo-usage-and-verbatim_shuffled_inf.csv', quotechar='"', index=False)

In [91]:
read_df = pd.read_csv('raw_data/TenantInfo-usage-and-verbatim_shuffled_inf.csv')

In [108]:
# read_df.head()

In [96]:
read_df_ = read_df.loc[:, ['Comment', 'Theme']]

In [102]:
read_df_.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161345 entries, 0 to 161724
Data columns (total 2 columns):
Comment    161345 non-null object
Theme      161345 non-null object
dtypes: object(2)
memory usage: 3.7+ MB


In [106]:
read_df.dropna(inplace=True)

In [107]:
read_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161345 entries, 0 to 161724
Columns: 230 entries, TenantId to AR_officeclient_06
dtypes: bool(40), float64(92), int64(62), object(36)
memory usage: 241.3+ MB


In [109]:
read_df.to_csv('raw_data/TenantInfo-usage-and-verbatim_shuffled_inf.csv', quotechar='"', index=False)